# Watson Crawler
In this notebook, I make a crawler for the news portal Watson (https://www.watson.ch/)
Goal is to get for every article
* title
* author
* publishing date
* number of comments
* themes
* article
So far I will do it only for the "Ressort Schweiz"

In [ ]:
# import the necessary libraries
import requests
from bs4 import BeautifulSoup
import xlwt
import time
from urllib.parse import urljoin
import csv

In [ ]:
# define classes 
class link_it(): # creates the new link for the sites
    def __init__(self,url, link):
        self.url = url
        self.link = urljoin(url,link)

class crawled_elements(): # stores the crawled elements
    def __init__(self, title, author, date, comments, themes, article):
        self.title = title
        self.author = author
        self.date = date
        self.comments = comments
        self.themes = themes
        self.article = article

In [ ]:
#the actual crawler
class Fetcher():
    def fetch(self,url):
        links = []
        while url != "":
            r = requests.get(url)
            time.sleep(2) 
            print(r)
            doc = BeautifulSoup(r.text, "html.parser")
            for row in doc.select(".teaserlink"):
                link = row.attrs["href"]
                l = link_it(url, link)
                links.append(l)
            print(l.link)
            # error exit or somewhat in the last line

            while True:
                try:
                    next_url = link_it(url, doc.select_one(".next a")["href"])
                    url = next_url.link
                    break
                except TypeError:
                    url = ""
                break


            # if doc.select_one(".next a")["href"] ==  None:
            #     url = ""
            # else:
            #     next_url = link_it(url, doc.select_one(".next a")["href"])
            #     url = next_url.link

        crawled = []
        for link in links:
            print(link.link)
            r = requests.get(link.link)
            time.sleep(2)
            doc = BeautifulSoup(r.text, "html.parser")
            while True:
                try:
                    title = doc.select_one(".maintitle").text
                    break
                except AttributeError:
                    title = "no_title"
                break
            print("title:", title)

            while True:
                try:
                    date = doc.select_one(".publish_date").text
                    break
                except AttributeError:
                    date = "no_date"
                break

            print("date:", date)
            while True:
                try:
                    author = doc.select_one(".card h6").text
                    break
                except AttributeError:
                    author = "no_author"
                break
            print("author:", author)
            while True:
                try:
                    comments = doc.select_one(".commentButton.standard-button .number").text
                    break
                except AttributeError:
                    comments = "no_comments"
                break
            print("comments:", comments)
            themes = []
            while True:
                try:
                    tags = doc.select(".widget.tags ul a")
                    for element in tags:
                        themes.append(element.text)
                    break
                except AttributeError:
                    themes.append("no_themes")
                break

            print("themes:", themes)
            article = []
            while True:
                try:
                    text = doc.select("p")
                    for paragraph in text:
                        article.append(paragraph.text)
                    break
                except AttributeError:
                    article.append("no_article")
                break
            print(article)
            crawled_article = crawled_elements(title,author,date,comments,themes,article)
            crawled.append(crawled_article)
        return(crawled)

In [ ]:
# to actually execute the crawler 
# be aware that it takes a while;)

x = Fetcher()
url = "https://www.watson.ch/Schweiz"
t = x.fetch(url)

with open("watson_schweiz.csv", "w", newline="", encoding='utf-8') as csvfile:
    countrywriter = csv.writer(csvfile, delimiter = ";",
                           quotechar='"', quoting=csv.QUOTE_MINIMAL)
    countrywriter.writerow(["title", "author", "date", "nmbr_comments", "themes", "article"])
    for article in t:
        countrywriter.writerow([article.title, article.author, article.date, article.comments, article.themes, article.article])